In [1]:
import csv 
import pandas as pd
import numpy as np

CSV File Paths

In [2]:
facilities_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/facilities.csv'
banks_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/banks.csv'
covenants_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/covenants.csv'
loans_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/loans.csv'

Using Pandas to read CSV files into Dataframes

In [3]:
DF_FACILITIES = pd.read_csv(facilities_path)
DF_BANKS = pd.read_csv(banks_path)
DF_COVENANTS = pd.read_csv(covenants_path)
DF_LOANS = pd.read_csv(loans_path)

In [4]:
DF_FACILITIES.head()

,amount,interest_rate,id,bank_id
0,1779071.0,0.04,5,1
1,23549.0,0.08,14,1
2,2127767.0,0.06,9,2
3,1991011.0,0.06,12,5
4,1938038.0,0.06,11,3


In [5]:
DF_LOANS.head()

,interest_rate,amount,id,default_likelihood,state
0,0.25,23549,1,0.04,LA
1,0.25,47821,2,0.05,TN
2,0.35,92388,3,0.07,SC
3,0.15,13961,4,0.01,OK
4,0.35,76811,5,0.07,WA


In [6]:
DF_COVENANTS.sort_values(by=['max_default_likelihood'])

,facility_id,max_default_likelihood,bank_id,banned_state
13,15,0.06,2,UT
22,6,0.08,5,KY
0,1,0.09,3,ND
1,5,0.09,1,LA
2,4,0.09,4,PA
3,7,0.09,5,RI
5,8,0.09,1,OK
7,11,0.09,3,TN
11,3,0.09,4,MA
20,2,0.09,4,TX


In [7]:
for attribute in DF_COVENANTS.columns:
    print('FEATURE:', attribute, ' UNIQUE VALUES:', len(DF_COVENANTS[attribute].unique()))

FEATURE: facility_id  UNIQUE VALUES: 15
FEATURE: max_default_likelihood  UNIQUE VALUES: 5
FEATURE: bank_id  UNIQUE VALUES: 5
FEATURE: banned_state  UNIQUE VALUES: 28


In [8]:
def create_assignment_table(LOAN_DATA, COVENANT_DATA):
    LOAN_DATA.sort_values(by = ['default_likelihood'], ascending = True)
    COVENANT_DATA = COVENANT_DATA.fillna(1).sort_values(by= ['max_default_likelihood'])
    DF_ASSIGNMENTS =  pd.DataFrame(columns=['loan_id', 'facility_id'])
    for index, loan in LOAN_DATA.iterrows():
        for index, covenant in COVENANT_DATA.iterrows():
            if (loan['default_likelihood'] < covenant['max_default_likelihood']) & (loan['state'] != covenant['banned_state']):
                DF_ASSIGNMENTS = DF_ASSIGNMENTS.append({'loan_id': loan['id'], 'facility_id':covenant['facility_id']}, ignore_index=True)
                break
    return(DF_ASSIGNMENTS)
df_test = create_assignment_table(DF_LOANS, DF_COVENANTS)
df_test.to_csv(path_or_buf = 'assignments_test.csv', index=False)

In [9]:
def get_expected_yeild(default_likelihood, loan_interest_rate, amount, facility_interest_rate):
    return (1 - default_likelihood) * loan_interest_rate * amount - default_likelihood * amount - facility_interest_rate * amount

In [10]:
def create_yeild_table(assigned_csv, LOAN_DATA, FACILITY_DATA):
    DF_ASSIGNED = pd.read_csv(assigned_csv)
    LOAN_DATA = LOAN_DATA.rename(columns = {'id':'loan_id', 'interest_rate':'loan_interest_rate'})
    FACILITY_DATA = FACILITY_DATA.rename(columns = {'id':'facility_id', 'amount':'facility_amount'})
    ASSIGNED_LOANS = pd.merge(DF_ASSIGNED, LOAN_DATA, on='loan_id')
    ALL_TOGETHER_NOW = pd.merge(FACILITY_DATA, ASSIGNED_LOANS, on = 'facility_id')
    DF_YIELD =  pd.DataFrame(columns=['facility_id', 'expected_yeild'])
    for index, assignment in ALL_TOGETHER_NOW.iterrows():
        facility_amount, interest_rate, facility_id, bank_id,loan_id,loan_interest_rate, amount, default_likelihood, state = assignment
        expected_yield = get_expected_yeild(default_likelihood, loan_interest_rate, amount, interest_rate)
        DF_YIELD = DF_YIELD.append({'facility_id': facility_id, 'expected_yield':expected_yield}, ignore_index=True)
    return DF_YIELD

In [11]:
yield_test = create_yeild_table('assignments_test.csv', DF_LOANS, DF_FACILITIES)

In [12]:
yield_test.groupby(['facility_id']).sum().reindex()

,expected_yeild
facility_id,
1.0,760885.6950
5.0,2252.5750
6.0,904258.6375
10.0,29556.8000
14.0,319350.2895
15.0,799956.7755
